In [1]:
import os
import pandas as pd
from tqdm import tqdm
from os import listdir
from os.path import isfile, join
import re
import numpy as np
import shutil

import openpyxl as xl
import json
from collections import OrderedDict
import soundfile as sf

In [ ]:
# 텍스트 파일 없는 DB 텍스트 추출

In [56]:
# 추출할 화자 입력
input_value = input()

 SnuMaleSadSplit


In [57]:
data_path = './SNU/'+input_value

In [58]:
with open(data_path+'/v0/alignment.json') as json_file:
    json_data = json.load(json_file)

In [59]:
def clean_text(text):
    pattern = '(\.wav)$'
    text = re.sub(pattern=pattern, repl='', string=text)
    text = text.strip()
    text = " ".join(text.split())
    return text  

In [60]:
files = listdir(data_path+'/v0/wavs')

In [61]:
fname_li = []

for i in files:
    i = clean_text(i)
    # female은 전체, male은 split 제외한 경우 
    #path = 'SNU/'+input_value+'/item/'+i
    # male에서 split된 경우
    path = 'SNU/'+'SnuMaleSad'+'/item/'+i
    fname_li.append(path)

In [62]:
json_data[fname_li[0]]

'내가 당신을 모르면 당신을 찾지도 않을테고'

In [63]:
text = []

for i in fname_li:
    j = json_data[i]
    text.append(j)

In [64]:
txt_df = pd.DataFrame(text)

In [65]:
txt_df.to_excel(data_path+'/v0/'+'transcription_'+input_value+'.xlsx', encoding='utf-8-sig', header=None)

In [44]:
#rr = pd.DataFrame(fname_li)

In [45]:
#rr.to_excel('./alignment_미반영.xlsx', encoding='utf-8-sig', header=None)

In [ ]:
### metadata.json 만들 db 선택 (ex:SNU)

In [2]:
input_value = input()

 SNU


In [3]:
data_path = './'+input_value
files = listdir(data_path)

In [13]:
for i in tqdm(files, mininterval=1):
    
    # 화자별 wav 파일리스트 생성
    # 만약 안에 wav 폴더 없으면 pass
    if os.path.isdir(data_path+"/"+i+'/v0/wavs'):
        wav_files = listdir(data_path+"/"+i+'/v0/wavs')
    
        # 음원파일의 duration, sample_rate, 파일명 경로, 스피커 구하기
        duration_li = []
        rate_li =[]
        fname_li = []
        spk_li = []
    
        for j in wav_files:
        
            # duration, sample_rate
            f = sf.SoundFile(data_path+"/"+i+'/v0/wavs/'+j)
            rate = f.samplerate
            dur = round((len(f) / f.samplerate),2)
            rate_li.append(rate)
            duration_li.append(dur)
    
            # 파일경로, speaker
            def clean_text(text):
                pattern = '(\.wav)$'
                text = re.sub(pattern=pattern, repl='', string=text)
                text = text.strip()
                text = " ".join(text.split())
                return text
        
            k = input_value+'/'+i+'/item/'+j
            k = clean_text(k)
            q = k.split('/')
            fname_li.append(k)
            spk_li.append(q[0]+'/'+q[1])
    
        # 텍스트 내용 불러오기
        txt_path = data_path+"/"+i+'/v0/transcription_'+i+'.xlsx'
        wb = xl.load_workbook(txt_path)
        ws = wb.active
    
        txt_li = []
        for r in ws.rows:
            txt_li.append(r[1].value)
    
        # df로 병합
        df = pd.DataFrame(fname_li, columns =['fname'])
        df.index += 1
        df['text'] = txt_li
        df['duration'] = duration_li
        df['speaker'] = spk_li
        df['sample_rate'] = rate_li
    
        # 데이터프레임 저장 (csv, xlsx)
        if not os.path.exists('./'+input_value+'_meta'):
            os.makedirs('./'+input_value+'_meta')

        df.to_csv('./'+input_value+'_meta/metadata_'+i+'.csv', encoding='utf-8-sig', header=None)
        df.to_excel('./'+input_value+'_meta/metadata_'+i+'.xlsx', encoding='utf-8-sig', header=None)
    
        # json 생성
        xlsx_path = './'+input_value+'_meta/metadata_'+i+'.xlsx'
        wb_2 = xl.load_workbook(xlsx_path)
        ws_2 = wb_2.active
        dic = OrderedDict()
    
        for z in ws_2.rows:
            dic[z[0].value] = {'fname':z[1].value,
                               'text':z[2].value,
                               'duration':z[3].value,
                               'speaker':z[4].value,
                               'sample_rate':z[5].value}
    
        with open('./'+input_value+'_meta/metadata_'+i+'.json', 'w', encoding="utf-8") as make_file:
            json.dump(dic, make_file, ensure_ascii=False, indent="\t")
    else:
        continue

100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [03:39<00:00,  9.14s/it]
